In [1]:
## SKIP THIS PART
# Ploomber related variables
upstream = None
product = None

In [ ]:
# Parameters
product = {"nb": "C:\\Users\\berkayg\\Desktop\\Coding env\\ test-case\\products\\clean_train_data.ipynb", "data": "C:\\Users\\berkayg\\Desktop\\Coding env\\ test-case\\products\\cleaned_train_data.csv"}


In [3]:
from src.data.config_reader import *
from src.data.path_finder import *
from src.data.database_configurations import *

import pandas as pd
import numpy as np

In [1]:
# Creating an SQLite database into which we can consolidate the seperate csv files as SQL tables
initiate_database()

In [5]:
# Access database
conn = connect_db()

## Data Manipulation and Cleaning

The aim is to transform the original train dataset into a binary classification suitable format. The original data consists only of the purchases __made__ by the users. In other words the data does not contain label "0". 

To overcome this problem, I have applied some manipulation techniques to join non-present **__currentbugroupname__** values to the original data; since non-presence indicates that the user did not purchase any thing with the given **__currentbugroupname__** label.

In [6]:
# Here we start manipulating the train dataset
df_target_train = pd.read_sql("SELECT * FROM df_target_train", conn)
df_target_train["userid"] = df_target_train["userid"].astype(str)
df_target_train.sort_values("userid", inplace=True)
df_target_train.dropna(subset=["currentbugroupname"], inplace=True)

In [7]:
# We now instantiate an empty DataFrame that will end up containing all the currentbugroupname combinations for each user
df_target_train_cleaned = pd.DataFrame()

# Storing unique userids and groupnames in order to create all "userid - currentbugroupname" scenarios
df_target_train_cleaned["userid"] = df_target_train["userid"].sort_values().unique()
df_target_train_cleaned["groupname"] = [df_target_train["currentbugroupname"].sort_values().unique().tolist()] * df_target_train_cleaned["userid"].nunique()
df_target_train_cleaned = df_target_train_cleaned.explode("groupname").sort_values("userid").reset_index(drop=True)
df_target_train_cleaned.sort_values(["userid"], inplace=True)
df_target_train_cleaned.reset_index(drop=True, inplace=True)

In [8]:
groupname_list = df_target_train["currentbugroupname"].sort_values().unique().tolist()

In [9]:
# After creating a synthetic train dataset we can gather the purchase information from the original data
# and use the information to label the corresponding rows as "1"
merged = pd.merge(df_target_train, df_target_train_cleaned, how='left', on=['userid'])
merged.sort_values("userid", inplace=True)

# Necessary operations to align the group names and purchases
# (i.e, if the user purchased an FMCG product the label will be "1")
merged["bol"] = merged.groupby(["userid", "currentbugroupname"]).currentbugroupname.transform(lambda x: x.duplicated())
merged["currentbugroupname"] = merged.apply(lambda x: np.nan if x["bol"] else x["currentbugroupname"], axis=1)
merged.reset_index(drop=True, inplace=True)
merged["y"] = merged.groupby("userid").apply(lambda x: x["groupname"].isin(x["currentbugroupname"])).reset_index()["groupname"].replace({True:1, False:0})

merged.drop(columns=["bol", "currentbugroupname"], inplace=True)
merged.drop_duplicates(subset=["userid", "groupname"], inplace=True)
merged.rename(columns={"groupname": "currentbugroupname"}, inplace=True)

In [10]:
# Notice that all 12 distinct groupnames are labeled accordingly for each user
merged.head(14)

,userid,probability,target,currentbugroupname,y
0,10000970,None,None,Aksesuar & Saat & Gözlük,1
1,10000970,None,None,Kozmetik,1
2,10000970,None,None,Mobilya,1
3,10000970,None,None,Private Label,1
4,10000970,None,None,FMCG,1
6,10000970,None,None,Ev,1
7,10000970,None,None,GAS,1
8,10000970,None,None,Ayakkabı & Çanta,1
9,10000970,None,None,Elektronik,1
10,10000970,None,None,GM,1


In [11]:
# Upon obtaining a suitable dataframe, we can start connecting other tables in the db
# To this end we will create a dictionary that will provide a valid currentbugroupname for each content in the database
df_products = pd.read_sql("SELECT contentid, currentbugroupname FROM df_product", conn)
df_products.head()

,contentid,currentbugroupname
0,40100964,Ayakkabı & Çanta
1,40427970,Branded Tekstil
2,41026644,FMCG
3,41227029,Branded Tekstil
4,40770080,Ev


In [12]:
products_dict = df_products.set_index("contentid").to_dict("dict")["currentbugroupname"]

In [13]:
# Now we can use this dictionary to create a currentbugroupname column in each table that contains contentid
m = merged.copy()

# We will use a for loop to iterate over the tables to link the contents to the group names and join the tables each other
for table in ["df_demo", "df_basket", "df_visit", "df_trx", "df_fav"]:
    df_table = pd.read_sql(f"SELECT * FROM {table}", conn)
    df_table["userid"] = df_table["userid"].astype(str)
    
    # Demographical data does not contain content, thus it will be handled with an else statement
    if table != "df_demo":
        df_table["currentbugroupname"] = df_table["contentid"].map(products_dict)
        df_table = df_table.groupby(["userid", "currentbugroupname"]).sum().reset_index().drop(columns=["contentid"])
        column_names = [f'{k}_{table.replace("df_", "")}' if k not in ["userid", "currentbugroupname"] else k for k in df_table.columns]
        df_table.columns = column_names
        m = m.merge(df_table, on=["userid", "currentbugroupname"], how="left")      
    else:
        df_table["gender"].str.replace("kvkktalepsilindi", "UNKNOWN")
        df_table["age"] = np.where(df_table["age"]>100, np.nan, df_table["age"])
        df_table["age"] = np.where(df_table["age"]<5, np.nan, df_table["age"])
        m = m.merge(df_table, on=["userid"], how="left")

In [14]:
# Table "m" seems more eligible for the task at hand now!
m.head()

,userid,probability,target,currentbugroupname,y,gender,age,tenure,addtobasket_count_basket,productdetailcount_visit,quantity_trx,price_trx,fav_count_fav
0,10000970,None,None,Aksesuar & Saat & Gözlük,1,Bayan,50.0,85,NaN,336.0,2.0,225.89,45.0
1,10000970,None,None,Kozmetik,1,Bayan,50.0,85,NaN,138.0,3.0,102.62,9.0
2,10000970,None,None,Mobilya,1,Bayan,50.0,85,NaN,NaN,NaN,NaN,NaN
3,10000970,None,None,Private Label,1,Bayan,50.0,85,NaN,19.0,NaN,NaN,2.0
4,10000970,None,None,FMCG,1,Bayan,50.0,85,NaN,4.0,NaN,NaN,NaN


In [15]:
# Imputing nan values
na_columns = m.select_dtypes(include=np.number).drop(columns=["age", "y"])
na_columns.fillna(0, inplace=True)
m.loc[:, na_columns.columns] = na_columns
m

,userid,probability,target,currentbugroupname,y,gender,age,tenure,addtobasket_count_basket,productdetailcount_visit,quantity_trx,price_trx,fav_count_fav
0,10000970,None,None,Aksesuar & Saat & Gözlük,1,Bayan,50.0,85,0.0,336.0,2.0,225.89,45.0
1,10000970,None,None,Kozmetik,1,Bayan,50.0,85,0.0,138.0,3.0,102.62,9.0
2,10000970,None,None,Mobilya,1,Bayan,50.0,85,0.0,0.0,0.0,0.00,0.0
3,10000970,None,None,Private Label,1,Bayan,50.0,85,0.0,19.0,0.0,0.00,2.0
4,10000970,None,None,FMCG,1,Bayan,50.0,85,0.0,4.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
244831,9998170,None,None,GAS,1,Bayan,24.0,85,0.0,0.0,0.0,0.00,0.0
244832,9998170,None,None,Digital Goods,1,Bayan,24.0,85,0.0,0.0,0.0,0.00,0.0
244833,9998170,None,None,Branded Tekstil,1,Bayan,24.0,85,0.0,233.0,0.0,0.00,0.0
244834,9998170,None,None,FMCG,1,Bayan,24.0,85,0.0,17.0,0.0,0.00,0.0


In [16]:
m.isnull().sum()

userid                           0
probability                 244836
target                      244836
currentbugroupname               0
y                                0
gender                           0
age                         140304
tenure                           0
addtobasket_count_basket         0
productdetailcount_visit         0
quantity_trx                     0
price_trx                        0
fav_count_fav                    0
dtype: int64

In [17]:
m.to_csv(product["data"], index=False)